In [38]:
from nlp_rake import rake
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
#from gensim.models import word2vec
import numpy as np
import pandas as pd

In [45]:
df_std = pd.read_csv('anom_detection/df_anom_std.csv')

In [46]:
dist_list = df_std[df_std['ST_NM'] == state]['0'].values.tolist()

In [47]:
dist_list = [word.lower() for word in dist_list]

In [66]:
stoppath = 'new_stoplist.txt'
#stoppath = 'SmartStoplist.txt'
#stoppath = 'FoxStoplist.txt'
rake_object = rake.Rake(stoppath, 5, 3, 4)


def onlySentences(article):
    new_article = ""
    for sentence in sent_tokenize(article):
        words_list = word_tokenize(sentence.lower())
        if len(set(dist_list).intersection(set(words_list)))>0:
            new_article += sentence 
            new_article += '\n'
    return new_article

def pureText(temp_article):
    article = temp_article[0].split('\n')[2]
    article += '\n'
    for article_full in temp_article[1:]:
        if len(article_full.split('\n'))> 1:
            article += article_full.split('\n')[3]
            article += '\n'
    return article

def articleDate(article):
    try:
        and_idx1 = article.find("|&| ")
        and_idx2 = article[and_idx1+4:].find("|&| ") + and_idx1 + 4
        and_idx3 = article.rfind("|&| ")

        article_date = article[and_idx2+4:and_idx3]

        dash_idx1 = article_date.find('-')

        dash_idx2 = article_date.rfind('-')

        year = int(article_date[:dash_idx1])
        month = int(article_date[dash_idx1+1:dash_idx2])
        day = int(article_date[dash_idx2+1:])

        return [article_date,year,month,day]

    except ValueError:
        pass

def do_lmtz(temp_article):
    lmtz = WordNetLemmatizer()
    words = word_tokenize(temp_article)
    new_article = ""
    for word in words:
        new_word = lmtz.lemmatize(word,'v')
        new_article = new_article + " " + new_word
    return new_article

def finalSet(article_list,start_idx,end_idx):
    
    text = pureText(article_list[start_idx:end_idx])
    sents = onlySentences(text)
    #lmtzd_sent = do_lmtz(sents)
    #lmtzd_text = do_lmtz(text)

    article = sents
    return article

def extractKeywords(article_list,start_idx,min_articles,number_keywords_want):
    articles_taken = min_articles

    end_idx = start_idx+articles_taken
    end_idx = min(end_idx,len(article_list)-1)

    article = finalSet(article_list,start_idx,end_idx)

    while(len(rake_object.run(article)) < number_keywords_want) and end_idx < len(article_list)-1:
        articles_taken += 1
        end_idx = start_idx+articles_taken

        article = finalSet(article_list,start_idx,end_idx)

    start_idx += articles_taken
    keywords = rake_object.run(article)
    print len(keywords)
    print
    keyword_0 = []
    for keyword in keywords:
        keyword_0.append(keyword[0])
        print keyword

    return [keywords,keyword_0]

def extractKeywords2(article_list,start_idx,end_idx):
    article = finalSet(article_list,start_idx,end_idx)
    print 'now sending the article to rake to extract keywords.... '
    keywords = rake_object.run(article)
    print 'now making the keywords arranged for you'
    #print keywords
    keyword_0 = []
    for keyword in keywords:
        keyword_0.append(keyword[0])
    return [keywords,keyword_0,article]


def articleList():
    date_ = str(year) + '-'
    if month<10:
        date_ += '0'+str(month)
    else:
        date_ += str(month)

    folder_name = state+'/txt_files/'
    f_name = folder_name+state + '_' + date_ + '.txt'
    f = open(f_name,'r')
    articles = f.read()

    article_list = articles.split('\n\n')
    return article_list

def countDays(article_list):
    curr_idx = 0
    count_dayAr = []
    for day in range(1,32):
        itr_day = day
        count_day = 0
        start_idx = curr_idx
        while(itr_day<day+1):
            curr_idx += 1
            if curr_idx>=len(article_list):
                break;
            article = article_list[curr_idx]
            if len(article) > 0:
                count_day += 1
                itr_day =  articleDate(article)[3]
        count_dayAr.append([day,count_day,start_idx,curr_idx])
    return count_dayAr



In [15]:
def heading(temp_article):
    article = temp_article[0].split('\n')[1]
    article += '\n'
    for article_full in temp_article[1:len(temp_article)-1]:
        article += article_full.split('\n')[2]
        article += '\n'
    return article

In [6]:
state = 'Uttar Pradesh'

In [7]:
year = 2012
month = 9

In [13]:
article_list = articleList()

In [20]:
article_heading = heading(article_list).split('\n')

In [25]:
article_heading

['Auto drivers to cripple Mumbai suburbs with twin rallies',
 'Don\xe2\x80\x99t levy tax on damages payout: Panel',
 "Man shot at, wife's body found killed and stuffed in bed box",
 'Row over book by senior Bengal cop',
 'Diagnostic centre staff assaults by patient\xe2\x80\x99s family',
 'Peacocks in NCR face Ranikhet virus threat',
 "Tattoo artist's body found in 16 pieces, cops say accident",
 'Crowd clashes with cops in Mayur Vihar-III, buses torched',
 'Garbage piles up in localities',
 'City hosps ready for flu challenge',
 'Lucknow hospitals ill-prepared to fight fire',
 'Monsoon recovery on,Sept off to a wet start',
 'Offering good package, foreign language courses in demand',
 'Body of missing woman found',
 'Man kills elder brother over property dispute',
 '\xe2\x80\x98Artificial crisis created to import coal at higher prices\xe2\x80\x99',
 'State government flooded withrequest for police gunners',
 "Foundation stone for UP's biggest dairy plant laid",
 "Historically, Goa's so

In [22]:
article = ''
for heading in article_heading:
    article += heading 

In [24]:
rake_object.run(article)

[('body found', 4.583333333333334),
 ('murder', 2.176470588235294),
 ('congress', 1.8888888888888888),
 ('akhilesh', 1.8181818181818181),
 ('death', 1.6666666666666667),
 ('rahul', 1.625),
 ('noida', 1.4615384615384615),
 ('house', 1.1666666666666667),
 ('contest', 1.0),
 ('india', 1.0)]

In [98]:
article_full

'\n'

In [93]:
len(temp_article)

1610

In [67]:
[keywords,keyword_0,article] = extractKeywords2(article_list,0,1600)

now sending the article to rake to extract keywords.... 
now making the keywords arranged for you


In [ ]:
article

In [68]:
keywords

[('generally cloudy sky', 8.916666666666666),
 ('gole dak khana', 8.666666666666668),
 ('arya samaj mandir', 8.657142857142858),
 ('kanya vidya dhan', 8.435897435897436),
 ('bhramaputra shopping complex', 8.333333333333334),
 ('latest india news', 8.183800713787335),
 ('presents art bazaar', 8.107142857142858),
 ('bahujan samaj party', 7.978609625668449),
 ('indian penal code', 7.903755868544602),
 ('halwara air base', 7.848252344416027),
 ('red ribbon express', 7.753968253968253),
 ('justice pks baghel', 7.7456445993031355),
 ('bharatiya janata party', 7.592895339954163),
 ('st columbas school', 7.516666666666667),
 ('foreign direct investment', 7.516129032258064),
 ('lok sabha elections', 7.437728937728938),
 ('safdarjang development area', 7.379279104381027),
 ('banaras hindu university', 7.373211353506919),
 ('dentist nupur talwar', 7.331236897274634),
 ('previous closing price', 7.325581395348838),
 ('escorts heart institute', 7.322388632872504),
 ('kotwali police station', 7.3147

In [69]:
keyword_0

['generally cloudy sky',
 'gole dak khana',
 'arya samaj mandir',
 'kanya vidya dhan',
 'bhramaputra shopping complex',
 'latest india news',
 'presents art bazaar',
 'bahujan samaj party',
 'indian penal code',
 'halwara air base',
 'red ribbon express',
 'justice pks baghel',
 'bharatiya janata party',
 'st columbas school',
 'foreign direct investment',
 'lok sabha elections',
 'safdarjang development area',
 'banaras hindu university',
 'dentist nupur talwar',
 'previous closing price',
 'escorts heart institute',
 'kotwali police station',
 'lok sabha election',
 'defaced religious book',
 'western air command',
 'electricity regulatory commission',
 'cmp degree college',
 'social forestry division',
 'public interest litigation',
 'atam shanti group',
 'cantonment railway station',
 'local intelligence unit',
 'dr sachans death',
 'special task force',
 'heavy rainfall warning',
 'public private partnership',
 'uninterrupted power supply',
 'power corporation limited',
 'nuclear 

In [87]:
len(article_list)

1610

In [32]:
countDays(article_list)

[[1, 1, 0, 1],
 [2, 7, 1, 8],
 [3, 13, 8, 21],
 [4, 5, 21, 26],
 [5, 9, 26, 35],
 [6, 14, 35, 49],
 [7, 8, 49, 57],
 [8, 19, 57, 76],
 [9, 8, 76, 84],
 [10, 6, 84, 90],
 [11, 10, 90, 100],
 [12, 8, 100, 108],
 [13, 17, 108, 125],
 [14, 16, 125, 141],
 [15, 11, 141, 152],
 [16, 16, 152, 168],
 [17, 6, 168, 174],
 [18, 6, 174, 180],
 [19, 6, 180, 186],
 [20, 17, 186, 203],
 [21, 10, 203, 213],
 [22, 5, 213, 218],
 [23, 11, 218, 229],
 [24, 13, 229, 242],
 [25, 10, 242, 252],
 [26, 15, 252, 267],
 [27, 19, 267, 286],
 [28, 7, 286, 293],
 [29, 11, 293, 304],
 [30, 11, 304, 315],
 [31, 10, 315, 327]]